- 병합된 데이터셋을 활용하여 스포츠 카테고리 용품만 추출. 시각화에 활용할 최종 데이터셋 구축

In [ ]:
import pandas as pd

file_path = "./custom_data/merge/order_customer_merged.csv"

order_df = pd.read_csv(file_path)

In [ ]:
# Filter the necessary columns
filtered_data = order_df[
    [
        "customer_city",
        "customer_state",
        "region",
        "product_category_name",
        "customer_id",
    ]
]

# Filter for specific product categories
esporte_lazer_data = filtered_data[
    filtered_data["product_category_name"] == "esporte_lazer"
]
fashion_esporte_data = filtered_data[
    filtered_data["product_category_name"] == "fashion_esporte"
]

In [ ]:
# 각 도시별 카테고리별 구매 고객 수
esporte_lazer_counts_city = (
    esporte_lazer_data.groupby(["customer_city"])["customer_id"]
    .count()
    .reset_index(name="esporte_lazer_customers_city")
)
fashion_esporte_counts_city = (
    fashion_esporte_data.groupby(["customer_city"])["customer_id"]
    .count()
    .reset_index(name="fashion_esporte_customers_city")
)

In [ ]:
# 병합
merged_df_city = pd.merge(
    esporte_lazer_counts_city,
    fashion_esporte_counts_city,
    on="customer_city",
    how="left",
)

In [ ]:
# 결측치 0으로 대체
merged_df_city.fillna(0, inplace=True)

In [ ]:
# 두 카테고리 합
merged_df_city["total"] = (
    merged_df_city["esporte_lazer_customers_city"]
    + merged_df_city["fashion_esporte_customers_city"]
)

In [ ]:
# 각 주별 카테고리별 구매 고객 수
esporte_lazer_counts_state = (
    esporte_lazer_data.groupby(["customer_state"])["customer_id"]
    .count()
    .reset_index(name="esporte_lazer_customers_state")
)
fashion_esporte_counts_state = (
    fashion_esporte_data.groupby(["customer_state"])["customer_id"]
    .count()
    .reset_index(name="fashion_esporte_customers_state")
)

In [ ]:
# 병합
merged_df_state = pd.merge(
    esporte_lazer_counts_state,
    fashion_esporte_counts_state,
    on="customer_state",
    how="left",
)

In [ ]:
# 결측치 0으로 대체
merged_df_state.fillna(0, inplace=True)

In [ ]:
# 두 카테고리의 합
merged_df_state["total"] = (
    merged_df_state["esporte_lazer_customers_state"]
    + merged_df_state["fashion_esporte_customers_state"]
)

In [ ]:
# 해안/내륙 컬럼 추가를 위한 리스트 생성
coastal_s = [
    "AP",
    "PA",
    "MA",
    "PI",
    "CE",
    "RN",
    "PB",
    "PE",
    "AL",
    "SE",
    "BA",
    "ES",
    "RJ",
    "SP",
    "PR",
    "SC",
    "RS",
]
inland_s = ["AM", "RR", "AC", "RO", "MT", "GO", "MG", "MS", "TO", "DF"]

# 주 코드에 해당하는 주 명칭 딕셔너리
state_mapping = {
    "AC": "Acre",
    "AL": "Alagoas",
    "AP": "Amapá",
    "AM": "Amazonas",
    "BA": "Bahia",
    "CE": "Ceará",
    "DF": "Distrito Federal",
    "ES": "Espírito Santo",
    "GO": "Goiás",
    "MA": "Maranhão",
    "MT": "Mato Grosso",
    "MS": "Mato Grosso do Sul",
    "MG": "Minas Gerais",
    "PA": "Pará",
    "PB": "Paraíba",
    "PR": "Paraná",
    "PE": "Pernambuco",
    "PI": "Piauí",
    "RN": "Rio Grande do Norte",
    "RJ": "Rio de Janeiro",
    "RS": "Rio Grande do Sul",
    "RO": "Rondônia",
    "RR": "Roraima",
    "SC": "Santa Catarina",
    "SP": "São Paulo",
    "SE": "Sergipe",
    "TO": "Tocantins",
}

# 각 주 코드에 해당하는 주 명칭 저장
coastal = [state_mapping[code] for code in coastal_s]
inland = [state_mapping[code] for code in inland_s]

In [ ]:
# 해안/내륙 컬럼에 데이터 추가
merged_df_state["region"] = merged_df_state["customer_state"].apply(
    lambda x: "coastal" if x in coastal else "inland"
)

In [ ]:
# csv 파일로 저장
merged_df_state.to_csv("state_sports_df.csv", index=False)

In [ ]:
# 해안/내륙 카테고리별 구매 고객 수
esporte_lazer_counts_region = (
    esporte_lazer_data.groupby(["region"])["customer_id"]
    .count()
    .reset_index(name="esporte_lazer_customers_region")
)
fashion_esporte_counts_region = (
    fashion_esporte_data.groupby(["region"])["customer_id"]
    .count()
    .reset_index(name="fashion_esporte_customers_region")
)

In [ ]:
# 병합
merged_df_region = pd.merge(
    esporte_lazer_counts_region, fashion_esporte_counts_region, on="region"
)

In [ ]:
# 두 카테고리의 합
merged_df_region["total"] = (
    merged_df_region["esporte_lazer_customers_region"]
    + merged_df_region["fashion_esporte_customers_region"]
)

In [ ]:
# csv 파일로 저장
merged_df_region.to_csv("region_sports_df.csv", index=False)